In [2]:
import vaex
import vaex.ml

import matplotlib.pyplot as plt


df = vaex.open('./helmi-dezeeuw-2000-FeH-v2_2.hdf5')
df

#,id,x,y,z,vx,vy,vz,E,L,Lz,FeH
0,25,1.5591525,-1.0355582,-1.2342551,-333.93762,14.448124,-127.04698,-120484.99,706.56744,-323.28503,-0.8045992
1,0,-1.2007784,-0.46247822,-0.35920778,251.57025,-230.97308,-57.01336,-155375.98,428.27997,393.69324,-0.95396304
2,7,-1.7470163,1.9494433,-2.0347395,-300.65564,209.71863,98.730156,-91698.15,1023.08856,219.72926,-1.796198
3,4,-5.90749,-1.2057447,3.5281541,-157.84805,-189.96689,200.09796,-79208.95,1201.368,931.903,-1.4722667
4,25,-4.6059566,-2.8173962,1.4604418,-19.82385,41.93743,5.5564847,-143025.4,260.63956,-249.01363,-0.89341164
...,...,...,...,...,...,...,...,...,...,...,...
"3,299,995",16,-10.562079,-13.517659,-0.255374,113.22222,15.439545,-27.64155,-82553.8,1454.4359,1367.4257,-2.0128398
"3,299,996",23,4.051899,1.5419586,-1.0596558,-223.5088,109.15839,-91.93414,-120274.94,995.62195,786.94006,-1.7964416
"3,299,997",22,2.5417993,1.9468315,-1.0098917,107.93812,33.782146,-84.91948,-157932.25,209.9375,-124.26989,-0.8232596
"3,299,998",27,6.1693034,4.3464093,7.2642264,33.274296,-72.55161,-84.66383,-102009.875,979.67175,-592.2166,-1.4459641


In [3]:
df.drop('id',inplace=True)

#,x,y,z,vx,vy,vz,E,L,Lz,FeH
0,1.5591525,-1.0355582,-1.2342551,-333.93762,14.448124,-127.04698,-120484.99,706.56744,-323.28503,-0.8045992
1,-1.2007784,-0.46247822,-0.35920778,251.57025,-230.97308,-57.01336,-155375.98,428.27997,393.69324,-0.95396304
2,-1.7470163,1.9494433,-2.0347395,-300.65564,209.71863,98.730156,-91698.15,1023.08856,219.72926,-1.796198
3,-5.90749,-1.2057447,3.5281541,-157.84805,-189.96689,200.09796,-79208.95,1201.368,931.903,-1.4722667
4,-4.6059566,-2.8173962,1.4604418,-19.82385,41.93743,5.5564847,-143025.4,260.63956,-249.01363,-0.89341164
...,...,...,...,...,...,...,...,...,...,...
"3,299,995",-10.562079,-13.517659,-0.255374,113.22222,15.439545,-27.64155,-82553.8,1454.4359,1367.4257,-2.0128398
"3,299,996",4.051899,1.5419586,-1.0596558,-223.5088,109.15839,-91.93414,-120274.94,995.62195,786.94006,-1.7964416
"3,299,997",2.5417993,1.9468315,-1.0098917,107.93812,33.782146,-84.91948,-157932.25,209.9375,-124.26989,-0.8232596
"3,299,998",6.1693034,4.3464093,7.2642264,33.274296,-72.55161,-84.66383,-102009.875,979.67175,-592.2166,-1.4459641


['Lz', 'FeH']

In [8]:
import vaex
import vaex.ml
from vaex.ml.sklearn import IncrementalPredictor
from sklearn.linear_model import SGDRegressor



features = df.column_names[:-2]
target = 'FeH'

standard_scaler = vaex.ml.StandardScaler(features=features)
df = standard_scaler.fit_transform(df)

features = df.get_column_names(regex='^standard')
model = SGDRegressor(learning_rate='constant', eta0=0.01, random_state=42)

incremental = IncrementalPredictor(model=model,
                                    features=features,
                                    target=target,
                                    batch_size=1000,
                                    num_epochs=3,
                                    shuffle=True,
                                    prediction_name='pred_FeH')
incremental.fit(df=df)

In [9]:
df = incremental.transform(df)
df.head(5)[['FeH', 'pred_FeH']]

#,FeH,pred_FeH
0,-0.804599,-0.804663
1,-0.953963,-0.954071
2,-1.7962,-1.79616
3,-1.47227,-1.47234
4,-0.893412,-0.893492


In [11]:
df.ml.metrics.mean_absolute_error(df['FeH'],df['pred_FeH'])

4.548781671780155e-05

In [12]:
df.ml.metrics.r2_score(df['FeH'],df['pred_FeH'])

0.9999999814282808